In [1]:
import os
import pickle
from tqdm import tqdm
import json

with open('arabic_laion_239006.pickle', 'rb') as f:
    data = pickle.load(f)

dir_path = './laion/part4/laion_5b_arabic_images'
files = os.listdir(dir_path)
captions = data['TEXT']
data_items = []

for file in tqdm(files):
    item = {}
    i = int(file[:-4])
    item['image_id'] = file
    item['caption'] = captions[i]
    data_items.append(item)

with open('./laion/data_part4.json', 'w') as f:
    json.dump(data_items, f)

100%|██████████| 14955/14955 [00:00<00:00, 450998.81it/s]


In [2]:
import torch
import skimage.io as io
import clip
from PIL import Image
import pickle
import json
import os
from tqdm import tqdm
import argparse


def main(clip_model_type: str):
    device = torch.device('cuda:0')
    clip_model_name = clip_model_type.replace('/', '_')
    out_path = f"./laion/laion_part4_{clip_model_name}_train.pkl"
    clip_model, preprocess = clip.load(clip_model_type, device=device, jit=False)
    with open('laion\data_part4.json', 'r') as f:
        data = json.load(f)
    print("%0d captions loaded from json " % len(data))
    all_embeddings = []
    all_captions = []
    for i in tqdm(range(len(data))):
        d = data[i]
        img_id = d["image_id"]
        filename = f"./laion/part4/laion_5b_arabic_images/{img_id}"
        image = io.imread(filename)
        image = preprocess(Image.fromarray(image)).unsqueeze(0).to(device)
        with torch.no_grad():
            prefix = clip_model.encode_image(image).cpu()
        d["clip_embedding"] = i
        all_embeddings.append(prefix)
        all_captions.append(d)
        if (i + 1) % 10000 == 0:
            with open(out_path, 'wb') as f:
                pickle.dump({"clip_embedding": torch.cat(all_embeddings, dim=0), "captions": all_captions}, f)

    with open(out_path, 'wb') as f:
        pickle.dump({"clip_embedding": torch.cat(all_embeddings, dim=0), "captions": all_captions}, f)

    print('Done')
    print("%0d embeddings saved " % len(all_embeddings))
    return 0


clip_model_type="ViT-B/32"
main(clip_model_type)

14955 captions loaded from json 


 73%|███████▎  | 10845/14955 [09:15<03:32, 19.34it/s]c:\Users\Edin\anaconda3\envs\mlp\lib\site-packages\PIL\TiffImagePlugin.py:858: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
 96%|█████████▌| 14291/14955 [12:10<00:33, 19.56it/s]c:\Users\Edin\anaconda3\envs\mlp\lib\site-packages\PIL\TiffImagePlugin.py:858: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 2. 
  warnings.warn(str(msg))
100%|██████████| 14955/14955 [12:44<00:00, 19.57it/s]

Done
14955 embeddings saved 


0

In [ ]:
from dataset import ClipGPTLaion5bArabicDataset

In [ ]:
ds = ClipGPTLaion5bArabicDataset('laion\laion_part4_ViT-B_32_train.pkl', 10)